In [ ]:
import os
import pandas as pd
pd.set_option('display.max_columns', 32)
pd.set_option('display.max_rows', 6)
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import healpy as hp
import numpy as np
from glob import glob
import itertools
import re

from vasttools.moc import VASTMOCS
from vasttools.pipeline import Pipeline

from mocpy import World2ScreenMPL, MOC
from astropy import units as u
from astropy.wcs import WCS
from astropy.visualization.wcsaxes.frame import EllipticalFrame
from astropy.coordinates import Angle, SkyCoord
from ligo.skymap.io import read_sky_map
from ligo.skymap.postprocess import find_greedy_credible_levels
from astropy.io import fits

'''GWTC1/2/3 skymaps obtained from:
https://dcc.ligo.org/LIGO-P1800381/public
https://dcc.ligo.org/LIGO-P2000223/public
https://zenodo.org/record/5546663#.YgNiR-pBwUE
respectively'''

In [ ]:
!pip install lxml

In [ ]:
vast_mocs= VASTMOCS()
vast_footprint = vast_mocs.load_pilot_epoch_moc('1')

def plot_moc(moc, name, cutoff, with_vast=False, savefig=False):
    fig = plt.figure(figsize=(24, 12), facecolor='white')
    
    with World2ScreenMPL(fig, 
                         fov=324 * u.deg,
                         center=SkyCoord(0, 0, unit='deg', frame='icrs'),
                         coordsys="icrs",
                         rotation=Angle(0, u.degree),
                         projection="AIT") as wcs:
        ax = fig.add_subplot(111, projection=wcs, frame_class=EllipticalFrame)
        ax.set_title(f"{name} Coverage with VAST")
        ax.grid(color="black", linestyle="dotted")
        
        if with_vast:
            vast_footprint.fill(ax=ax, wcs=wcs, alpha=0.9, fill=True, linewidth=0, color="#00bb00", label='VAST Footprint', zorder = 0)
            vast_footprint.border(ax=ax, wcs=wcs, alpha=0.5, color="black")
        
        moc.fill(ax=ax, wcs=wcs, alpha=0.9, fill=True, linewidth=0, color="red", label=f"{name} {cutoff} probability area", zorder = 1)
        moc.border(ax=ax, wcs=wcs, alpha=0.5, color="black")
        
        ax.legend()
    plt.show()
    plt.close()
    cutoff = str(cutoff).replace(".", "-")
    if savefig:
        fig_name = f"{name}_{cutoff}"
        if os.path.exists(fig_name+'.png'):
            fig_name += '_(1)'
        fig.savefig(fig_name+'.png', facecolor=fig.get_facecolor())

In [ ]:
def process_event(skymap,
                  event_name,
                  cutoffs,
                  plot=False,
                  moc=None,
                  with_vast=True,
                  savefig=False,
                  save_significant=True
                 ):
    coverages = []
    areas = []
    
    for cutoff in cutoffs:
        skymap = hp.ud_grade(skymap, coverage_NSIDE, order_in='NESTED', power=-2)
        nside = hp.get_nside(skymap)
        credible_levels = find_greedy_credible_levels(skymap)
        idx = np.where(credible_levels < cutoff)[0]
        overlap = skymap[idx] * coverage_map[idx]
        coverage = np.sum(overlap)
        area = np.sum(credible_levels[np.where(overlap)]) * hp.nside2pixarea(nside, degrees=True)
            
        coverages.append(coverage)
        areas.append(area)

        if save_significant and coverage >= 0.5:
            level = np.log2(nside)
            levels = np.ones(len(idx)) * level
            # Significant coverage
            if not moc:
                moc = MOC.from_healpix_cells(idx, depth=levels)
            plot_moc(moc, event_name, cutoff, with_vast, savefig=True)

        elif plot:
            level = np.log2(nside)
            levels = np.ones(len(idx)) * level
            if not moc:
                moc = MOC.from_healpix_cells(idx, depth=levels)
            plot_moc(moc, event_name, cutoff, with_vast, savefig)
    return coverages, areas, moc

In [ ]:
def plot_all_covered(
    skymap,
    event_name,
    with_vast=True,
    savefig=False,
    save_significant=True
    ):

    skymap = hp.ud_grade(skymap, coverage_NSIDE, order_in='NESTED', power=-2)
    nside = hp.get_nside(skymap)
    credible_levels = find_greedy_credible_levels(skymap)
    idx = np.where(credible_levels < cutoff)[0]
    level = np.log2(nside)
    levels = np.ones(len(idx)) * level
    moc = MOC.from_healpix_cells(idx, depth=levels)
    plot_moc(moc, event_name, cutoff, with_vast, savefig)
    return moc

In [ ]:
footprint_file = 'full_VAST_footprint_1024.fits'
coverage_map = hp.read_map(footprint_file, nest=True)
coverage_NPIX= len(coverage_map)
coverage_NSIDE = hp.npix2nside(coverage_NPIX)
hp.mollview(coverage_map, nest=True)

In [ ]:
%matplotlib inline
data = []
cutoffs = [0.9]

cutoff_header = [str(int(x*100))+"_percent_coverage" for x in cutoffs]
area_header = [str(int(x*100))+"_percent_area" for x in cutoffs]
columns = ['Event'] + [x for x in itertools.chain.from_iterable(zip(cutoff_header, area_header))]

files = glob('./GWTC1/*') + glob('./GWTC2/*PublicationSamples.fits') + glob('./GWTC3/*Mixed.fits')

In [ ]:
for i, f in enumerate(files[:]):
    event_name = re.findall("(GW[0-9]{6}_[0-9]{6}|GW[0-9]{6})", f)[0]
    skymap, history = read_sky_map(f, nest=True)
#     hp.mollview(skymap, nest=True)
    coverages, areas = process_event(skymap, event_name, cutoffs, plot=True, savefig=False, save_significant=False)
    result = [event_name] + [x for x in itertools.chain.from_iterable(zip(coverages, areas))]
    data.append(result)
    print(f"{i+1}/{len(files)}: {result}")

df = pd.DataFrame(data, columns=columns).sort_values('90_percent_coverage', ascending=False)
df.to_csv('coverages.csv', index=False)

In [ ]:
# Plots GW events with >0.5 pdf coverage over VAST footprint
%matplotlib inline
df = pd.read_csv('coverages.csv')
well_covered = df[df['90_percent_coverage'] >= 0.5]
well_covered

union_skymap = None

mocs = []
for i, f in enumerate(files[:]):
    event_name = re.findall("(GW[0-9]{6}_[0-9]{6}|GW[0-9]{6})", f)[0]
    if event_name in well_covered['Event'].values:
        print(event_name)
        skymap, history = read_sky_map(f, nest=True)
        skymap = hp.ud_grade(skymap, coverage_NSIDE, order_in='NESTED', power=-2)
        nside = hp.get_nside(skymap)
        credible_levels = find_greedy_credible_levels(skymap)
        idx = np.where(credible_levels < 0.9)[0]
        level = np.log2(nside)
        levels = np.ones(len(idx)) * level
        moc = MOC.from_healpix_cells(idx, depth=levels)
        mocs.append((event_name, moc))

In [ ]:
fig = plt.figure(figsize=(24, 12), facecolor='white')

with World2ScreenMPL(fig, 
                     fov=324 * u.deg,
                     center=SkyCoord(0, 0, unit='deg', frame='icrs'),
                     coordsys="icrs",
                     rotation=Angle(0, u.degree),
                     projection="AIT") as wcs:
    ax = fig.add_subplot(111, projection=wcs, frame_class=EllipticalFrame)
    ax.set_title(f"Coverage with VAST")
    ax.grid(color="black", linestyle="dotted")

    vast_footprint.fill(ax=ax, wcs=wcs, alpha=0.2, fill=True, linewidth=0, color="#00bb00", label='VAST Footprint', zorder = 0)
    vast_footprint.border(ax=ax, wcs=wcs, alpha=0.8, color="black")

    color = cm.rainbow(np.linspace(0, 1, len(mocs)))

    for ((name, moc), i) in zip(mocs, color):
        moc.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, linewidth=0, color=i, label=name)
        moc.border(ax=ax, wcs=wcs, alpha=0.9, color='black')
    plt.tight_layout()
    ax.legend()

plt.savefig('significant_coverages.png', dpi=200)

# AMON Coverages

In [ ]:
pages = [
    'https://gcn.gsfc.nasa.gov/amon_icecube_gold_bronze_events.html',
#     'https://gcn.gsfc.nasa.gov/amon_hawc_events.html',
#     'https://gcn.gsfc.nasa.gov/amon_nu_em_coinc_events.html',
#     'https://gcn.gsfc.nasa.gov/amon_icecube_cascade_events.html'
]

cols = ['RunNum_EventNum', 'Rev', 'RA [deg]', 'RA', 'Dec [deg]', 'Dec', 'Error90 [arcmin]', 'Error90', 'Error50 [arcmin]', 'Error50', 'Error']

NSIDE=2**10
NPIX= hp.nside2npix(NSIDE)
# Takes a while to run this line ~2 minute
#lon, lat = zip(*map(lambda x: hp.pix2ang(NSIDE, x, nest=True, lonlat=True), range(NPIX)))

In [ ]:
%matplotlib inline
data = []
cutoffs = [0.9]

cutoff_header = [str(int(x*100))+"_percent_coverage" for x in cutoffs]
area_header = [str(int(x*100))+"_percent_area" for x in cutoffs]
columns = ['Event'] + [x for x in itertools.chain.from_iterable(zip(cutoff_header, area_header))]

for page in pages:
    df = pd.read_html(page)[0]
    df.columns = df.columns.droplevel()
    events = df[df.columns.intersection(cols)]
    display(events)
    
for idx, event in enumerate(events.iterrows()):
    event_name = event[1].RunNum_EventNum

    m_90 = MOC.from_cone(event[1]['RA [deg]']*u.deg,
                         event[1]['Dec [deg]']*u.deg,
                         event[1]['Error90 [arcmin]']*u.arcmin,
                         10)
#     m_50 = MOC.from_cone(event[1]['RA [deg]']*u.deg,
#                          event[1]['Dec [deg]']*u.deg,
#                          event[1]['Error50 [arcmin]']*u.arcmin,
#                          10)

    no_overlap = MOC.empty(vast_footprint.intersection(m_90)) 
    coverages, areas = [0.0], [0.0]
    if not no_overlap:
        moc_hp = m_90.contains(lon*u.deg, lat*u.deg)
        hp_map = np.where(moc_hp, 1, 0)
        coverages, areas = process_event(hp_map, event_name, cutoffs, percent_covered=True, plot=False, moc=m_90, savefig=False, save_significant=False)
    result = [event_name] + [x for x in itertools.chain.from_iterable(zip(coverages, areas))]
    data.append(result)
    print(f"{idx+1}/{events.shape[0]}")

df = pd.DataFrame(data, columns=columns).sort_values('90_percent_coverage', ascending=False)
df.to_csv('amon_coverages.csv', index=False)

In [ ]:
for page in pages:
    df = pd.read_html(page)[0]
    df.columns = df.columns.droplevel()
    events = df[df.columns.intersection(cols)]
    display(events)

In [ ]:
revs = list(set(events[events.Rev == 0].RunNum_EventNum.values))
len(revs)

In [ ]:
rev_events = events[((events.RunNum_EventNum.isin(revs)) & (events.Rev == 0) | (~events.RunNum_EventNum.isin(revs)))].reset_index(drop=True)
rev_events

In [ ]:
folder = 'AMON'
for idx, event in enumerate(rev_events.iterrows()):
    event_name = event[1].RunNum_EventNum

    m_90 = MOC.from_cone(event[1]['RA [deg]']*u.deg,
                         event[1]['Dec [deg]']*u.deg,
                         event[1]['Error90 [arcmin]']*u.arcmin,
                         10)

    no_overlap = MOC.empty(vast_footprint.intersection(m_90)) 
    
    if not no_overlap:
        m_90.write(os.path.join(folder, event_name+'.fits'))